# Проект по SQL
- Задача — проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры - - книг. 

# Задание
- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

## Подключение к базе

In [17]:
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
  'pwd': 'Sdf4$2;d-d30pp', # пароль
  'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
  'port': 6432, # порт подключения
  'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})


## Исследование таблиц

### Исследуем таблицу books

In [18]:
# Формируем sql-запрос.
query = ''' SELECT *
            FROM books
            ORDER BY publication_date DESC
            Limit 5
        '''
pd.io.sql.read_sql(query, con = engine) 

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,43,377,A Quick Bite (Argeneau #1),360,2020-03-31,28
1,635,166,The Art of Loving,192,2019-08-06,130
2,445,612,Monster,281,2019-03-05,14
3,293,80,Ham on Rye,288,2014-07-29,99
4,993,80,Women,291,2014-07-29,99


In [19]:
# Формируем sql-запрос.
query = ''' SELECT *
            FROM books
        '''
books = pd.io.sql.read_sql(query, con = engine) 
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


### Исследуем таблицу ratings

In [20]:
# Формируем sql-запрос.
query = ''' SELECT *
            FROM ratings
            Limit 5
        '''
pd.io.sql.read_sql(query, con = engine) 

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [21]:
# Формируем sql-запрос.
query = ''' SELECT *
            FROM ratings
        '''
ratings = pd.io.sql.read_sql(query, con = engine) 
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


### Исследуем таблицу reviews

In [22]:
# Формируем sql-запрос.
query = ''' SELECT *
            FROM reviews
            LIMIT 5
        '''
pd.io.sql.read_sql(query, con = engine) 

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [23]:
# Формируем sql-запрос.
query = ''' SELECT *
            FROM reviews
        '''
reviews = pd.io.sql.read_sql(query, con = engine) 
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


Проверяем, может ли приходится 2 автора на 1 ревью 

In [24]:
# Формируем sql-запрос.
query = ''' SELECT review_id, count(username)
            FROM reviews
            group by review_id
            HAVING count(username)>1
            '''
pd.io.sql.read_sql(query, con = engine) 

,review_id,count


Проверяем, может ли приходится 1 автор на несколько ревью

In [25]:
# Формируем sql-запрос.
query = ''' SELECT username, count(review_id)
            FROM reviews
            group by username
            HAVING count(review_id)>1
            '''
pd.io.sql.read_sql(query, con = engine) 

,username,count
0,zjohnston,14
1,jnelson,21
2,lewisdesiree,19
3,patrickhudson,15
4,ulowe,16
...,...,...
155,shermannatalie,24
156,dmiller,18
157,charlesraymond,16
158,isaiahreyes,15


Проверяем, может ли приходится несколько ревью на 1 книгу

In [26]:
# Формируем sql-запрос.
query = ''' SELECT book_id, count(review_id)
            FROM reviews
            group by book_id
            HAVING count(review_id)>1
            '''
pd.io.sql.read_sql(query, con = engine) 

,book_id,count
0,652,2
1,273,2
2,51,5
3,951,2
4,839,4
...,...,...
952,64,4
953,55,2
954,148,3
955,790,2


### Исследуем таблицу publishers

In [27]:
query = ''' SELECT *
            FROM publishers
            Limit 5
        '''
pd.io.sql.read_sql(query, con = engine) 

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [28]:
# Формируем sql-запрос.
query = ''' SELECT *
            FROM publishers
        '''
publishers = pd.io.sql.read_sql(query, con = engine) 
publishers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


### Исследуем таблицу authors

In [29]:
query = ''' SELECT *
            FROM authors
            Limit 5
        '''
pd.io.sql.read_sql(query, con = engine) 

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [30]:
# Формируем sql-запрос.
query = ''' SELECT *
            FROM authors
        '''
publishers = pd.io.sql.read_sql(query, con = engine) 
publishers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


## Задание

Посчитайте, сколько книг вышло после 1 января 2000 года;


In [31]:
query = ''' SELECT count(book_id)
            FROM books
            Where publication_date >'2000-01-01'
            '''

pd.io.sql.read_sql(query, con = engine)


,count
0,819


После 1 января 2000 года вышло 819 книг

Для каждой книги посчитайте количество обзоров и среднюю оценку

In [32]:
query = ''' WITH
            R AS (SELECT r.book_id as book, count(r.review_id) as review
            FROM reviews as r
            GROUP BY r.book_id),
            
            
            RA AS (SELECT ra.book_id as book, AVG(ra.rating) as rating
            FROM ratings as ra
            GROUP BY ra.book_id)
            
            
            SELECT b.book_id, R.review, RA.rating
            FROM  books as b
            LEFT JOIN R
            ON b.book_id=R.book
            LEFT JOIN RA
            ON b.book_id=RA.book
            ORDER BY R.review DESC NULLS LAST
            
       '''
pd.io.sql.read_sql(query, con = engine)

,book_id,review,rating
0,948,7.0,3.662500
1,854,6.0,3.772727
2,627,6.0,3.789474
3,695,6.0,4.081081
4,733,6.0,3.750000
...,...,...,...
995,672,NaN,5.000000
996,191,NaN,4.000000
997,221,NaN,4.000000
998,83,NaN,3.666667


In [33]:
query = ''' SELECT COUNT(DISTINCT r.review_id) 
            FROM reviews as r
           
       '''
pd.io.sql.read_sql(query, con = engine)

,count
0,2793


In [34]:
query = ''' SELECT COUNT(DISTINCT r.review_id) 
            FROM reviews as r
           
       '''
pd.io.sql.read_sql(query, con = engine)

,count
0,2793


Посчитали кол-во обзоров для каждой книги. Максимальное кол-во=7, минимальное=1.Максимальная оценка=5, минимальная=1,5. Не на все книги есть информация об отзывах

Определите издательство, которое выпустило наибольшее число книг толще
50 страниц — так вы исключите из анализа брошюры;


In [148]:
query = '''WITH
            H AS (SELECT p.publisher, count(b.book_id) as count
            FROM publishers as p
            LEFT OUTER JOIN books as b
            ON p.publisher_id=b.publisher_id
            WHERE b.num_pages>50
            GROUP BY p.publisher_id)
           
           SELECT h.publisher
           FROM H
           WHERE count=(SELECT max(count) FROM H)
           
       '''
pd.io.sql.read_sql(query, con = engine)

,publisher
0,Penguin Books


Издательство, которое выпустило наибольшее число книг толще 50 страниц,- Penguin Books

Определите автора с самой высокой средней оценкой книг — учитывайте
только книги с 50 и более оценками;


In [149]:
query = '''WITH
            B AS (SELECT b.book_id as book,
            count(ra.username) as count
            
            FROM books as b
            LEFT OUTER JOIN ratings as ra
            ON b.book_id=ra.book_id
            GROUP BY b.book_id
            HAVING count(ra.username)>=50)
            
            SELECT a.author
            FROM authors as a
            INNER JOIN books as b
            ON  a.author_id=b.author_id
            INNER JOIN ratings as ra
            ON b.book_id=ra.book_id
            WHERE b.book_id in (SELECT B.book FROM B)
            GROUP BY a.author
            ORDER BY avg(ra.rating) desc
            LIMIT 1
            
          
       '''
pd.io.sql.read_sql(query, con = engine)

,author
0,J.K. Rowling/Mary GrandPré


Автор с самой высокой средней оценкой книг — учитывайте
только книги с 50 и более оценками,-J.K. Rowling/Mary GrandPré


Посчитайте среднее количество обзоров от пользователей, которые поставили
больше 50 оценок.

In [150]:
query = '''WITH
            N AS (SELECT ra.username as user,
            count(ra.rating) as count
            
            FROM ratings as ra
            GROUP BY ra.username
            HAVING count(ra.rating)>50)
            
            SELECT AVG(M.count) 
            FROM
            (SELECT r.username as user,COUNT(r.review_id) as count
            FROM reviews as r
            WHERE r.username in (SELECT N.user FROM N)
            GROUP BY r.username) as M
          
       '''
pd.io.sql.read_sql(query, con = engine)

,avg
0,24.333333


Cреднее количество обзоров от пользователей, которые поставили
больше 50 оценок,-24,3

# Вывод

- Мы имеем данные по следующим таблицам: books,review,ratings,authors,publishers
- Из имеющихся данных мы знаем:
- После 1 января 2000 года вышло 819 книг
- кол-во обзоров для каждой книги и средний рейтинг. Максимальное кол-во=7, минимальное=1.Максимальная оценка=5, - - - - - -    - минимальная=1,5. Не на все книги есть информация об отзывах.
- издательство, которое выпустило наибольшее число книг толще 50 страниц,- Penguin Books
- автор с самой высокой средней оценкой книг — учитывайте
- только книги с 50 и более оценками,-J.K. Rowling/Mary GrandPré
- среднее количество обзоров от пользователей, которые поставили
- больше 50 оценок,-24,3